In [2]:
!pip install tensorflow-datasets --quiet
!pip install pydot --quiet
!pip install transformers --quiet

     |████████████████████████████████| 5.5 MB 5.2 MB/s 
     |████████████████████████████████| 182 kB 55.7 MB/s 
     |████████████████████████████████| 7.6 MB 64.2 MB/s 


In [3]:
import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds

import sklearn as sk
import os
import nltk
from nltk.data import find

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

#let's make longer output readable without scrolling
from pprint import pprint
import re

In [4]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# define paths in colab
dataset_path = 'drive/MyDrive/Colab Notebooks/w266_project_data'
csv_path = 'drive/MyDrive/Colab Notebooks/w266_project_predictions/'
model_path = 'drive/MyDrive/Colab Notebooks/w266_project_models/'

In [6]:
# a list of Davidson's translated model output CSV files
output_file_names = ['davidson_t5_test_output.csv', 'davidson_xsum_output.csv', 'davidson_bart_cnn_test.csv', 
                     'davidson_paradetox_output.csv']

In [7]:
# a list of Davidson's classification model CSV files
class_file_names = ['davidson_bertweet_class_pred.csv', 'davidson_bert_class_pred.csv', 'davidson_bertlarge_class_pred.csv']

## Models

In [8]:
max_length = 65

### BERT Tweet Model

#### load BERT Tweet pretrain

In [9]:
model_checkpoint = 'vinai/bertweet-base'

In [10]:
def create_bert_model(checkpoint = model_checkpoint,
                                hidden_size = 66, 
                                dropout=0.3,
                                learning_rate=0.00005):
  """
  Build a simple classification model with BERT. Use the Pooled Output for classification purposes.
  """
  #bert_model = TFBertModel.from_pretrained(checkpoint)                                              
  ### YOUR CODE HERE

  input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='input_ids_layer')
  token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='token_type_ids_layer')
  attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='attention_mask_layer')

  bert_inputs = {'input_ids': input_ids,
  'token_type_ids': token_type_ids,
  'attention_mask': attention_mask
  }      

  bert_out = bertweet_model(bert_inputs)
  pooled_token = bert_out[1]

  hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(pooled_token)
  hidden = tf.keras.layers.Dropout(dropout, name='dropout_layer')(hidden)

  classification = tf.keras.layers.Dense(3, activation='softmax',name='classification_layer')(hidden)

  classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])

  classification_model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
      metrics=['accuracy'])


  ### END YOUR CODE
  return classification_model

In [11]:
from transformers import AutoModel, AutoTokenizer, TFAutoModel

bertweet_model = TFAutoModel.from_pretrained(model_checkpoint)
bertweet_tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False)

Downloading:   0%|          | 0.00/558 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/740M [00:00<?, ?B/s]

Some layers from the model checkpoint at vinai/bertweet-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at vinai/bertweet-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Downloading:   0%|          | 0.00/843k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### load fine-tuned BERTWEET model (to compare with fine-tuned balanced BERTWEET model 

In [12]:
model_name = 'davidson_bertweet_weights.hdf5'

pooled_bertweet_model = create_bert_model()

# The model weights (that are considered the best) are loaded into the
# model.
checkpoint_file = model_path + model_name
pooled_bertweet_model.load_weights(checkpoint_file)

#### load Balanced fine-tuned BERTWEET model

In [13]:
model_name = 'davidson_bertweet_balanced_weights.hdf5'
balanced_bertweet_model = create_bert_model()

checkpoint_file = model_path + model_name
balanced_bertweet_model.load_weights(checkpoint_file)

### Load T5, XSUM, CBB and Paradetox Translated Data and run the BERTWEET classification model

In [14]:
class_file_name = 'davidson_bertweet_class_pred.csv'
df_class = pd.read_csv(csv_path + class_file_name)
df_class

,org_class,org_input,pred_class
0,1,"""Why would you wanna be the Green Ranger? He's...",0
1,2,#HolySpirit God still share HIS #Secrets Amos ...,2
2,2,pancakes trash,2
3,2,"The KFAN mock draft continues, Cleveland is ""o...",2
4,1,I be telling Mcgirt music ain't enough.You got...,0
...,...,...,...
2474,2,Slack jawed yokel husband http://t.co/VE1PWFrz9t,0
2475,1,Dating you would be like Darnell dating that f...,1
2476,2,Did you say spray tan? **Charlie Crist switche...,2
2477,1,"bitches be like "" I'm a squirter"" but thinkin ...",1


### Davidson Org. Input Classification

In [15]:
# run model.predict()
bertweet_encodings = bertweet_tokenizer(list(df_class['org_input']), truncation=True, padding='max_length', max_length=max_length, return_tensors='tf')
un_predictions = pooled_bertweet_model.predict([bertweet_encodings.input_ids, bertweet_encodings.token_type_ids, bertweet_encodings.attention_mask])
un_prediction_classes = un_predictions.argmax(axis=-1)
balanced_predictions = balanced_bertweet_model.predict([bertweet_encodings.input_ids, bertweet_encodings.token_type_ids, bertweet_encodings.attention_mask])
balanced_prediction_classes = balanced_predictions.argmax(axis=-1)

balanced_prediction_classes = balanced_prediction_classes[:2400]

print('unbalanced = ', np.unique(un_prediction_classes, return_counts=True))
print('balanced = ', np.unique(balanced_prediction_classes, return_counts=True))

#balanced_prediction_classes

78/78 [==============================] - 13s 132ms/step
unbalanced =  (array([0, 1, 2]), array([ 461, 1996,   22]))
balanced =  (array([0, 1, 2]), array([ 747, 1237,  416]))


#### Other Models Classification

In [ ]:
# a list of Davidson's translated model output CSV files
output_file_names = ['davidson_t5_test_output.csv', 'davidson_xsum_output.csv', 
                     'davidson_bart_cnn_test.csv', 'davidson_paradetox_output.csv']
result_file_names = ['davidson_bertweet_class_outoput_t5.csv','davidson_bertweet_class_outoput_xsum.csv',
                     'davidson_bertweet_class_outoput_cnn.csv','davidson_bertweet_class_outoput_paradetox.csv']

In [ ]:
for i in range(len(output_file_names)):
  #print(result_file_names[i])
  output_file_name = output_file_names[i]
  result_file_name = result_file_names[i]

  df_predictions = pd.read_csv(csv_path + output_file_name)

  # run model.predict()
  bertweet_encodings = bertweet_tokenizer(list(df_predictions['test_predictions']), truncation=True, padding='max_length', max_length=max_length, return_tensors='tf')
  un_predictions = pooled_bertweet_model.predict([bertweet_encodings.input_ids, bertweet_encodings.token_type_ids, bertweet_encodings.attention_mask])
  un_prediction_classes = un_predictions.argmax(axis=-1)
  balanced_predictions = balanced_bertweet_model.predict([bertweet_encodings.input_ids, bertweet_encodings.token_type_ids, bertweet_encodings.attention_mask])
  balanced_prediction_classes = balanced_predictions.argmax(axis=-1)

  print(output_file_name)
  print('unbalanced = ', np.unique(un_prediction_classes, return_counts=True))
  print('balanced = ', np.unique(balanced_prediction_classes, return_counts=True))

  df_class = df_class[:2400]
  df_predictions = df_predictions[:2400]
  balanced_prediction_classes = balanced_prediction_classes[:2400]
  
  # save prediction output to CSV file
  save_csv_dict = {'org_class': df_class['org_class'], 'org_input': df_class['org_input'], 'pred_class': df_class['pred_class'], 
                 'output': df_predictions['test_predictions'], 'output_class': balanced_prediction_classes}
  df_save = pd.DataFrame(save_csv_dict) 

  # saving the output dataframe to a csv file
  df_save.to_csv(csv_path + result_file_name, index = False) 
  print()



75/75 [==============================] - 10s 139ms/step
davidson_t5_test_output.csv
unbalanced =  (array([0, 1, 2]), array([1359, 1007,   34]))
balanced =  (array([0, 1, 2]), array([ 511,  381, 1508]))

75/75 [==============================] - 10s 133ms/step
davidson_xsum_output.csv
unbalanced =  (array([0, 1, 2]), array([1527,  841,   32]))
balanced =  (array([0, 1, 2]), array([ 411,  281, 1708]))

78/78 [==============================] - 11s 145ms/step
davidson_bart_cnn_test.csv
unbalanced =  (array([0, 1, 2]), array([1708,  731,   31]))
balanced =  (array([0, 1, 2]), array([ 461,  233, 1776]))

78/78 [==============================] - 11s 142ms/step
davidson_paradetox_output.csv
unbalanced =  (array([0, 1, 2]), array([1654,  771,   45]))
balanced =  (array([0, 1, 2]), array([ 421,  252, 1797]))



### Read CVS files, Display all cases for each CSV file

In [ ]:
result_file_names = ['davidson_bertweet_class_outoput_t5.csv','davidson_bertweet_class_outoput_xsum.csv',
                     'davidson_bertweet_class_outoput_cnn.csv','davidson_bertweet_class_outoput_paradetox.csv']

In [ ]:
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [ ]:
for i in range(len(result_file_names)):
  result_file_name = result_file_names[i]
  df_results = pd.read_csv(csv_path + result_file_name)
  df_results['class_type'] = df_results['pred_class'].map(str) + df_results['output_class'].map(str)
  print('\033[1m' + result_file_name + '\033[0m')
  print()
  print('class comparsion type counts')
  #print('class type counts = ', np.unique(df_results['class_type'], return_counts=True))
  df_group = df_results.groupby(list(df_results['class_type'])).size().reset_index()
  df_group.rename({0:'count'}, inplace=True, axis=1)
  display(df_group.T)
  print()

  # print first 10 examples in all cases 
  print('org_input = 0, output = 0')
  display(df_results.loc[df_results['class_type'] == '00', ['org_input', 'output']].head(10))
  print('org_input = 0, output = 1')
  display(df_results.loc[df_results['class_type'] == '01', ['org_input', 'output']].head(10))
  print('org_input = 0, output = 2')
  display(df_results.loc[df_results['class_type'] == '02', ['org_input', 'output']].head(10))
  print('org_input = 1, output = 0')
  display(df_results.loc[df_results['class_type'] == '10', ['org_input', 'output']].head(10))
  print('org_input = 1, output = 1')
  display(df_results.loc[df_results['class_type'] == '11', ['org_input', 'output']].head(10))
  print('org_input = 1, output = 2')
  display(df_results.loc[df_results['class_type'] == '12', ['org_input', 'output']].head(10))
  print('org_input = 2, output = 0')
  display(df_results.loc[df_results['class_type'] == '20', ['org_input', 'output']].head(10))
  print('org_input = 2, output = 1')
  display(df_results.loc[df_results['class_type'] == '21', ['org_input', 'output']].head(10))
  print('org_input = 2, output = 2')
  display(df_results.loc[df_results['class_type'] == '22', ['org_input', 'output']].head(10))
  print()
  print('==========================================================')
  print()


davidson_bertweet_class_outoput_t5.csv

class comparsion type counts


,0,1,2,3,4,5,6,7
index,00,01,02,10,11,12,21,22
count,424,43,280,87,336,814,2,414



org_input = 0, output = 0


,org_input,output
0,"""Why would you wanna be the Green Ranger? He's evil!"" Duh bitch",Why would you wanna be the Green Ranger? He's evil
6,"Hittin licks, what you get outta that?\nYou &amp; yo boys is some pussy cats!!!","Hittin licks, what you get outta that? You &amp; yo boys is a bad person"
12,real shiit my niggah ;;,real shiit my niggah ;;
25,Ugly bitches stay subbing,Ugly girls stay subbing
26,"""..the nig..""? ok, you're the big time shit talker on twitter no name, no face, no info","""..the nig..""? ok, you're the big time talker on twitter no name, no face, no info"
27,"""Fuck me or I'll make you feel bad about it. But if you fuck other guys, I'll call you a hoe. And if you fuck me, I'll thi;","""Fuck me or I'll make you feel bad about it. But if you fuck other guys, I'll call you a hoe. And if you fuck me, I'll thi;"
28,I don't like a ghetto female;;;,I don't like a ghetto female;;;
30,All these girls saying nobody can put em up with em. Dont no nigga want a bitch as a girlfriend fix that shit,All these girls saying nobody can put em up with em. Dont no nigga want a girl as a girlfriend fix that
31,"""Searching"" the incredible story of 1 white man's quest to find the ingredients necessary to make Caribbean Jerked Chicken in redneck land","""Searching"" the incredible story of 1 white man's quest to find the ingredients necessary to make Caribbean Jerked Chicken in redneck land"
38,What a fucking faggot,What a faggot! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! !


org_input = 0, output = 1


...

,org_input,output
583,Free wop,Free
878,"Good even-ink, honkies!",Good even-ink honkies!
2155,And I don't have any confidence NONWHATSOEVER in you Barack! You're the sole reason why this country is in thi;,And I don't have any confidence in you Barack! You're the sole reason why this country is in thi;


org_input = 2, output = 1


,org_input,output
1292,"Superman your ho, eyyyy",Superman your ho


org_input = 2, output = 2


,org_input,output
1,#HolySpirit God still share HIS #Secrets Amos 3:7 blessings #Jesus,God still share HIS #Secrets Amos 3:7 blessings #Jesus
2,pancakes trash,Pancakes are not good.
3,"The KFAN mock draft continues, Cleveland is ""on the clock"". Paul Allen has the Vikings 8th pick. I am going with Def back, Justin Gilbert","The KFAN mock draft continues, Cleveland is ""on the clock"". Paul Allen has the Vikings 8th pick."
10,lmao girlllaaaa I don't like em chunky anymore.;;;;;,I don't like them chunky anymore.
16,"Cactus is a yellow lab chow mix too! Hershey was everybody's ""active"" buddy. Cactus is one special pup. She's an Angel.","Cactus is a yellow lab chow mix too! Hershey was everybody's ""active"" buddy. Cactus is one special pup. She's an Angel"
22,New jersey should be yellow with a green chevron stripe like charlie brown's jersey,New jersey should be yellow with a green chevron stripe like charlie brown's jersey
24,"Peyton Manning's remarkable season:\n; 5,477 Pass yds (NFL record)\n; 55 Pass TD (NFL record)\n; 450 completions\n; likely ho;","Peyton Manning's remarkable season: 5,477 Pass yds (NFL record) Gleaming; 450 completions Gleaming; likely ho;"
33,. Because Outman is a brand-new #Yankees and because Girardi will never use 3 relievers when he can use 6.,Because Outman is new and because Girardi will never use 3 relievers when he can use 6.
34,Jeter almost broke Yankee stadium,Jeter almost broke Yankee stadium
45,the bird with the aim of dreams https://t.co/jkX6dsqXcq,The bird with the aim of dreams.


In [ ]:
df_results = pd.read_csv(csv_path + result_file_name)

In [ ]:
df_results

,org_class,org_input,pred_class,output,output_class
0,1,"""Why would you wanna be the Green Ranger? He's...",0,Why would you wanna be the Green Ranger? He's ...,0
1,2,#HolySpirit God still share HIS #Secrets Amos ...,2,#HolySpirit God still share HIS #Secrets Amos ...,2
2,2,pancakes trash,2,pancakes are not good,2
3,2,"The KFAN mock draft continues, Cleveland is ""o...",2,"The KFAN mock draft continues, Cleveland is ""o...",2
4,1,I be telling Mcgirt music ain't enough.You got...,0,I be telling Mcgirt music ain't enough.You got...,2
...,...,...,...,...,...
2395,1,tell me why nick cannon bout to invite wiz to ...,1,tell me why nick cannon bout to invite wiz to ...,2
2396,1,"I just wanna touch da bitch, fuck da bitch, Li...",1,"I just wanna touch her, don't trust her.",2
2397,2,i dont think women should wear flesh tone colo...,2,i dont think women should wear flesh tone colo...,2
2398,1,You put nair on your pussy for the gram? Ok ht...,1,You put nair on you for the gram? Ok http://t....,2


### coding practices

In [ ]:
df_results['class_type'] = df_results['pred_class'].map(str) + df_results['output_class'].map(str)

In [ ]:
df_results

,org_class,org_input,pred_class,output,output_class,class_type
0,1,"""Why would you wanna be the Green Ranger? He's...",0,Why would you wanna be the Green Ranger? He's ...,0,00
1,2,#HolySpirit God still share HIS #Secrets Amos ...,2,#HolySpirit God still share HIS #Secrets Amos ...,2,22
2,2,pancakes trash,2,pancakes are not good,2,22
3,2,"The KFAN mock draft continues, Cleveland is ""o...",2,"The KFAN mock draft continues, Cleveland is ""o...",2,22
4,1,I be telling Mcgirt music ain't enough.You got...,0,I be telling Mcgirt music ain't enough.You got...,2,02
...,...,...,...,...,...,...
2395,1,tell me why nick cannon bout to invite wiz to ...,1,tell me why nick cannon bout to invite wiz to ...,2,12
2396,1,"I just wanna touch da bitch, fuck da bitch, Li...",1,"I just wanna touch her, don't trust her.",2,12
2397,2,i dont think women should wear flesh tone colo...,2,i dont think women should wear flesh tone colo...,2,22
2398,1,You put nair on your pussy for the gram? Ok ht...,1,You put nair on you for the gram? Ok http://t....,2,12


In [ ]:
unique, counts = np.unique(df_results['class_type'], return_counts=True)
#display(np.asarray((unique, counts)).T)
np.vstack((unique, counts)).T

array([['00', 290],
       ['01', 36],
       ['02', 421],
       ['10', 115],
       ['11', 210],
       ['12', 912],
       ['20', 3],
       ['21', 1],
       ['22', 412]], dtype=object)

In [ ]:
df_group = df_results.groupby(list(df_results['class_type'])).size().reset_index()
df_group.rename({0:'count'}, inplace=True, axis=1)
df_group.T

,0,1,2,3,4,5,6,7,8
index,00,01,02,10,11,12,20,21,22
count,290,36,421,115,210,912,3,1,412


In [ ]:
# print first 10 examples in all cases 
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

print('org_input = 0, output = 0')
display(df_results.loc[df_results['class_type'] == '00', ['org_input', 'output']].head(10))
print('org_input = 0, output = 1')
display(df_results.loc[df_results['class_type'] == '01', ['org_input', 'output']].head(10))
print('org_input = 0, output = 2')
display(df_results.loc[df_results['class_type'] == '02', ['org_input', 'output']].head(10))
print('org_input = 1, output = 0')
display(df_results.loc[df_results['class_type'] == '10', ['org_input', 'output']].head(10))
print('org_input = 1, output = 1')
display(df_results.loc[df_results['class_type'] == '11', ['org_input', 'output']].head(10))
print('org_input = 1, output = 2')
display(df_results.loc[df_results['class_type'] == '12', ['org_input', 'output']].head(10))
print('org_input = 2, output = 0')
display(df_results.loc[df_results['class_type'] == '20', ['org_input', 'output']].head(10))
print('org_input = 2, output = 1')
display(df_results.loc[df_results['class_type'] == '21', ['org_input', 'output']].head(10))
print('org_input = 2, output = 2')
display(df_results.loc[df_results['class_type'] == '22', ['org_input', 'output']].head(10))

org_input = 0, output = 0


,org_input,output
0,"""Why would you wanna be the Green Ranger? He's evil!"" Duh bitch",Why would you wanna be the Green Ranger? He's evil
4,I be telling Mcgirt music ain't enough.You gotta have a non music related agenda.Them crackers want to sell something with your face.,I be telling Mcgirt music ain't enough.You gotta have a non music related agenda.Them crackers want to sell something with your face
25,Ugly bitches stay subbing,Ugly girls stay subbing
31,"""Searching"" the incredible story of 1 white man's quest to find the ingredients necessary to make Caribbean Jerked Chicken in redneck land","""Searching"" the incredible story of 1 white man's quest to find the ingredients necessary to make Caribbean Jerked Chicken in redneck land"
38,What a fucking faggot,What a bad person
40,He is really on my line singing negro spirituals..!!,He is really on my line singing negro spirituals
48,bitch u ugly then hoe,You are ugly
51,This bitch just called me a bougie whore tho! ;;;; ol angry bird! Lmao,This person just called me a bougie tho!
69,"NO, you don?t have ?haters?. People just don?t like you. Get over yourself hoe.","NO, you don't have?haters?. People just don't like you. Get over yourself"
86,I'm not saying I'm for this but .. Studs will fuck trannys but won't let they bitch strap them down you muhfuckas childish lol,I'm not saying I'm for this but .. Studs will hurt trannys but won't let them strap them down


org_input = 0, output = 1


,org_input,output
28,I don't like a ghetto female;;;,I don't like a ghetto female
30,All these girls saying nobody can put em up with em. Dont no nigga want a bitch as a girlfriend fix that shit,All these girls saying nobody can put em up with them. Don't want a girlfriend fix that.
43,"""i wish i was born in a different generation , cause yall just make my pussy bones hurt . sicken ass children .;","i wish i was born in a different generation , cause yall just make my pussy bones hurt."
44,"I was disgusted, betrayed, hurt, &amp; my pussy drier than a niggas skin in the winter time","I was disappointed, betrayed, hurt, &amp; my pussy drier than a funny skin in the winter time"
71,Greta... You let a fag do this to me I thought we were cool,Greta... You let a girl do this to me I thought we were cool
163,"all them ""model"" hoes follow a nigga...""blow up""...an then hit the infamous imma only follow 25 people wave...","All of them ""model"" hoes follow a guy...blow up""...an then hit the infamous imma only follow 25 people wave..."
212,"Fuck your dreads fire it up hoe rt while we watch ""This Is The End""","your dreads fire it up hoe rt while we watch ""This Is The End"""
293,After ya girl walks in ya crib and has hoes for ya niggas. http://t.co/lE1r8tiwD1,After ya girl walks in ya crib and has hoes for ya.
317,"How I'm supposed to know you not just a butch dyke if yeen got a beard, my man?","How I'm supposed to know you not just a butch if you got a beard, my man?"
331,I wasn't serious bitch ; bitch them niggas i named ain't fuccing witcho ass ;; we all just making shit up ;;,I wasn't serious. The people I named ain't fuccing witcho. We're just making things up.


org_input = 0, output = 2


...

,org_input,output
583,Free wop,Free
878,"Good even-ink, honkies!",Good even-ink honkies!
2155,And I don't have any confidence NONWHATSOEVER in you Barack! You're the sole reason why this country is in thi;,And I don't have any confidence in you Barack! You're the sole reason why this country is in thi;


org_input = 2, output = 1


,org_input,output
1292,"Superman your ho, eyyyy",Superman your ho


org_input = 2, output = 2


,org_input,output
1,#HolySpirit God still share HIS #Secrets Amos 3:7 blessings #Jesus,God still share HIS #Secrets Amos 3:7 blessings #Jesus
2,pancakes trash,Pancakes are not good.
3,"The KFAN mock draft continues, Cleveland is ""on the clock"". Paul Allen has the Vikings 8th pick. I am going with Def back, Justin Gilbert","The KFAN mock draft continues, Cleveland is ""on the clock"". Paul Allen has the Vikings 8th pick."
10,lmao girlllaaaa I don't like em chunky anymore.;;;;;,I don't like them chunky anymore.
16,"Cactus is a yellow lab chow mix too! Hershey was everybody's ""active"" buddy. Cactus is one special pup. She's an Angel.","Cactus is a yellow lab chow mix too! Hershey was everybody's ""active"" buddy. Cactus is one special pup. She's an Angel"
22,New jersey should be yellow with a green chevron stripe like charlie brown's jersey,New jersey should be yellow with a green chevron stripe like charlie brown's jersey
24,"Peyton Manning's remarkable season:\n; 5,477 Pass yds (NFL record)\n; 55 Pass TD (NFL record)\n; 450 completions\n; likely ho;","Peyton Manning's remarkable season: 5,477 Pass yds (NFL record) Gleaming; 450 completions Gleaming; likely ho;"
33,. Because Outman is a brand-new #Yankees and because Girardi will never use 3 relievers when he can use 6.,Because Outman is new and because Girardi will never use 3 relievers when he can use 6.
34,Jeter almost broke Yankee stadium,Jeter almost broke Yankee stadium
45,the bird with the aim of dreams https://t.co/jkX6dsqXcq,The bird with the aim of dreams.


In [ ]:
match df_results['class_type']:
    case '00':
         action-1
    case pattern-2:
         action-2
    case pattern-3:
         action-3
    case _:
        action-default